<a href="https://colab.research.google.com/github/Nnamchisimi/Nnamchisimi/blob/main/(use)Backorder_and_updated_stckdateetc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

updated, for bckorder and rejected , empty stckentrydate(12/05/2024)

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (2).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (2).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces and unwanted characters
if 'Customer info' in df.columns:
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[^A-Za-z0-9]', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[^A-Za-z0-9]', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[^A-Za-z0-9]', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        # Check for each DN number in the body
        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    return tracking_dict

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
df['status'] = df.apply(determine_status, axis=1)

# Update status for rejected items
df.loc[(df['Items status'] == 'Rejected') & (df['status'] == 'backorder'), 'status'] = 'Rejected'

# Create the list_of_backorders column for searching
def determine_backorders(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.isnull(inv_date_dispatch_date) and pd.isnull(conf_dd_confirmed_dispatch_date) and (pd.notnull(expected_dd_expected_to_be_dispatched) or pd.isnull(expected_dd_expected_to_be_dispatched)):
        return 'backorders'
    else:
        return ''

# Apply the function to create the new list_of_backorders column
df['list_of_backorders'] = df.apply(determine_backorders, axis=1)

# Filter out 'Rejected' items when searching for backorders
filtered_df = df[(df['status'] == 'backorder') & (df['list_of_backorders'] == 'backorders')]

# Get the list of DN numbers from the DataFrame
dn_numbers = df['DN no.'].dropna().astype(str).tolist()

# Search for tracking numbers in emails
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Update the existing entries with the tracking numbers
df['Tracking No'] = df['DN no.'].map(tracking_dict)

# Create the tracking details column
df['tracking_details'] = df['Tracking No'].apply(lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else '')

# Set tracking_details to blank if the value contains 'nan'
df['tracking_details'] = df['tracking_details'].apply(lambda x: '' if 'nan' in x else x)

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")

# Close the connection and logout
mail.close()
mail.logout()


Enter the customer info: girne
Customer info Details:
       ±     gATP  Code Cust. No.  Cust. order no.              Part number  \
127   ⊞    Green   C21   CY822025         42871473  A 222 905 5111     9051   
128   ⊞    Green   C21   CY822025         42871473  A 222 905 0309     9051   
129   ⊞    Green   C21   CY822025         42871473  A 213 905 4803     9051   
617   ⊞   Yellow   C22   CY822025         42621444           A 222 899 0600   
618   ⊞    Green         CY822025         42621444           A 270 200 0200   
619   ⊞    Green         CY822025         42621444           A 205 330 5801   
620   ⊞    Green         CY822025         42621444           A 002 990 2017   
621   ⊞    Green         CY822025         42621444           A 210 501 0615   
622   ⊞    Green         CY822025         42621444           A 654 094 0204   
624   ⊞    Green         CY822025         42621444           A 169 540 1617   
635   ⊞   Yellow   C22   CY822025         42621444           A 222 899 0600  

KeyboardInterrupt: 

does not recognize tags behind customer info (fixed) 13/08/2024

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (11).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (11).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1= '/content/df1_unique.xlsx'
exdf2= '/content/df2.xlsx'
df1.to_excel(exdf1, index=False)
df2.to_excel(exdf2, index=False)
# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-mt'
if 'Customer info' in df.columns:
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'-[A-Z]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'-[A-Z]+$', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'-[A-Z]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        # Check for each DN number in the body
        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    return tracking_dict

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
df['status'] = df.apply(determine_status, axis=1)

# Update status for rejected items
df.loc[(df['Items status'] == 'Rejected') & (df['status'] == 'backorder'), 'status'] = 'Rejected'

# Create the list_of_backorders column for searching
def determine_backorders(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.isnull(inv_date_dispatch_date) and pd.isnull(conf_dd_confirmed_dispatch_date) and (pd.notnull(expected_dd_expected_to_be_dispatched) or pd.isnull(expected_dd_expected_to_be_dispatched)):
        return 'backorders'
    else:
        return ''

# Apply the function to create the new list_of_backorders column
df['list_of_backorders'] = df.apply(determine_backorders, axis=1)

# Filter out 'Rejected' items when searching for backorders
filtered_df = df[(df['status'] == 'backorder') & (df['list_of_backorders'] == 'backorders')]

# Get the list of DN numbers from the DataFrame
dn_numbers = df['DN no.'].dropna().astype(str).tolist()

# Search for tracking numbers in emails
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Update the existing entries with the tracking numbers
df['Tracking No'] = df['DN no.'].map(tracking_dict)

# Create the tracking details column
df['tracking_details'] = df['Tracking No'].apply(lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else '')

# Set tracking_details to blank if the value contains 'nan'
df['tracking_details'] = df['tracking_details'].apply(lambda x: '' if 'nan' in x else x)

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")

# Close the connection and logout
mail.close()
mail.logout()


Enter the customer info: girne
Customer info Details:
       ±     gATP  Code Cust. No.  Cust. order no.              Part number  \
624   ⊞    Green   C21   CY822025         42871473  A 222 905 5111     9051   
625   ⊞    Green   C21   CY822025         42871473  A 222 905 0309     9051   
626   ⊞    Green   C21   CY822025         42871473  A 213 905 4803     9051   
1114  ⊞   Yellow   C22   CY822025         42621444           A 222 899 0600   
1115  ⊞    Green         CY822025         42621444           A 270 200 0200   
1116  ⊞    Green         CY822025         42621444           A 205 330 5801   
1117  ⊞    Green         CY822025         42621444           A 002 990 2017   
1118  ⊞    Green         CY822025         42621444           A 210 501 0615   
1119  ⊞    Green         CY822025         42621444           A 654 094 0204   
1121  ⊞    Green         CY822025         42621444           A 169 540 1617   
1132  ⊞   Yellow   C22   CY822025         42621444           A 222 899 0600  

('BYE', [b'LOGOUT Requested'])

*** WORKS CONFIRMED 20/01/25 USE THIS Updated does not recognize tags behind customer info including -i and /G (fixed) 10/09/2024  can also be used for the new logbus output fıle ef update_order_items.  (can also be used  for the new logbus sıte, take sameıfles updated _order and merc_yedek_parca 07_01_25)***    

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (43).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (43).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1 = '/content/df1_unique.xlsx'
exdf2 = '/content/df2.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
if 'Customer info' in df.columns:
    # Updated regex to handle various suffixes including special characters
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Save df1_unique for verification
    df1_unique.to_excel(exdf1, index=False)


    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        # Check for each DN number in the body
        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    return tracking_dict

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
df['status'] = df.apply(determine_status, axis=1)

# Update status for rejected items
df.loc[(df['Items status'] == 'Rejected') & (df['status'] == 'backorder'), 'status'] = 'Rejected'

# Create the list_of_backorders column for searching
def determine_backorders(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.isnull(inv_date_dispatch_date) and pd.isnull(conf_dd_confirmed_dispatch_date) and (pd.notnull(expected_dd_expected_to_be_dispatched) or pd.isnull(expected_dd_expected_to_be_dispatched)):
        return 'backorders'
    else:
        return ''

# Apply the function to create the new list_of_backorders column
df['list_of_backorders'] = df.apply(determine_backorders, axis=1)

# Filter out 'Rejected' items when searching for backorders
filtered_df = df[(df['status'] == 'backorder') & (df['list_of_backorders'] == 'backorders')]

# Get the list of DN numbers from the DataFrame
dn_numbers = df['DN no.'].dropna().astype(str).tolist()

# Search for tracking numbers in emails
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Update the existing entries with the tracking numbers
df['Tracking No'] = df['DN no.'].map(tracking_dict)

# Create the tracking details column
df['tracking_details'] = df['Tracking No'].apply(lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else '')

# Set tracking_details to blank if the value contains 'nan'
df['tracking_details'] = df['tracking_details'].apply(lambda x: '' if 'nan' in x else x)

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")

# Close the connection and logout
mail.close()
mail.logout()


Enter the customer info: Girne
Customer info Details:
       ±    gATP  Code Cust. No.  Cust. order no.     Part number  \
183   ⊞   Green         CY822025         44051755  A 270 159 0600   
184   ⊞   Green         CY822025         44051755  A 004 159 5803   
185   ⊞   Green         CY822025         44051755  A 271 016 1221   
186   ⊞   Green         CY822025         44051755  A 271 016 1321   
189   ⊞     Red   C21   CY822025         44051755  A 004 994 1845   
...   ..     ...   ...       ...              ...             ...   
3178  ⊞   Green         CY822025         43281568  A 223 835 2300   
4086  ⊞   Green         CY822025         43041533  A 270 200 0200   
4095  ⊞   Green         CY822025         43041533  A 222 899 0600   
4103  ⊞   Green         CY822025         43041533  A 447 880 1114   
4104  ⊞   Green         CY822025         43041533  A 447 880 1214   

     Designation of part no. Ordered Designation of part no. Confirmed  \
183                       SPARK PLUG       

('BYE', [b'LOGOUT Requested'])

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (38).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (38).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1 = '/content/df1_unique.xlsx'
exdf2 = '/content/df2.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
if 'Customer info' in df.columns:
    # Updated regex to handle various suffixes including special characters
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Save df1_unique for verification
    df1_unique.to_excel(exdf1, index=False)

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        # Check for each DN number in the body
        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    return tracking_dict

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
df['status'] = df.apply(determine_status, axis=1)

# Update status for rejected items
df.loc[(df['Items status'] == 'Rejected') & (df['status'] == 'backorder'), 'status'] = 'Rejected'

# Create the list_of_backorders column for searching
def determine_backorders(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.isnull(inv_date_dispatch_date) and pd.isnull(conf_dd_confirmed_dispatch_date) and (pd.notnull(expected_dd_expected_to_be_dispatched) or pd.isnull(expected_dd_expected_to_be_dispatched)):
        return 'backorders'
    else:
        return ''

# Apply the function to create the new list_of_backorders column
df['list_of_backorders'] = df.apply(determine_backorders, axis=1)

# Filter out 'Rejected' items when searching for backorders
filtered_df = df[(df['status'] == 'backorder') & (df['list_of_backorders'] == 'backorders')]

# Get the list of DN numbers from the DataFrame
dn_numbers = df['DN no.'].dropna().astype(str).tolist()

# Search for tracking numbers in emails
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Update the existing entries with the tracking numbers
df['Tracking No'] = df['DN no.'].map(tracking_dict)

# Create the tracking details column
df['tracking_details'] = df['Tracking No'].apply(lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else '')

# Set tracking_details to blank if the value contains 'nan'
df['tracking_details'] = df['tracking_details'].apply(lambda x: '' if 'nan' in x else x)

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")

# Close the connection and logout
mail.close()
mail.logout()

Enter the customer info: girne
Customer info Details:
       ±     gATP  Code Cust. No.  Cust. order no.              Part number  \
5     ⊞   Yellow         CY822025         43931734          N 000000 004039   
122   ⊞    Green         CY822025         43931734           A 447 420 6600   
123   ⊞    Green         CY822025         43931734           A 447 420 6700   
124   ⊞    Green         CY822025         43931734           A 654 094 1500   
133   ⊞    Green         CY822025         43931734           A 002 990 2017   
147   ⊞    Green         CY822025         43931734           A 906 540 1517   
155   ⊞    Green         CY822025         43931734  A 247 584 3313     Z102   
156   ⊞      Red         CY822025         43931734  A 206 584 6505     Z102   
157   ⊞      Red         CY822025         43931734           A 243 814 0000   
158   ⊞    Green         CY822025         43931734  A 177 680 8002     9G33   
159   ⊞      Red         CY822025         43931734  A 247 680 0203     9G33  

KeyboardInterrupt: 

***code for new star order site 16_01_25  trıal stıll ***

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (38).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (38).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1 = '/content/df1_unique.xlsx'
exdf2 = '/content/df2.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
if 'Customer info' in df.columns:
    # Updated regex to handle various suffixes including special characters
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Function to get customer info details
def get_customer_info_details():
    customer_info_input = input("Enter the customer info: ").strip()
    if 'Cleaned Customer info' in df_cleaned.columns:
        part_details = df_cleaned[df_cleaned['Cleaned Customer info'].astype(str).str.contains(customer_info_input, case=False, na=False)]
    else:
        print("The column 'Cleaned Customer info' was not found in the cleaned file.")
        return

    if not part_details.empty:
        print("Customer info Details:")
        print(part_details)
    else:
        print("No details found for the entered customer info.")

# Run the function
get_customer_info_details()

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Check for required columns in df1 and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the first file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1 and df2
    df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1
    df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
    df1_unique = df1.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Save df1_unique for verification
    df1_unique.to_excel(exdf1, index=False)

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Connect to the email server
mail = imaplib.IMAP4_SSL('imap.gmail.com')

def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            print(f"Failed to fetch email with ID {email_id}")
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        # Decode email body
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        # Check for each DN number in the body
        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    return tracking_dict

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
df['status'] = df.apply(determine_status, axis=1)

# Update status for rejected items
df.loc[(df['Items status'] == 'Rejected') & (df['status'] == 'backorder'), 'status'] = 'Rejected'

# Create the list_of_backorders column for searching
def determine_backorders(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.isnull(inv_date_dispatch_date) and pd.isnull(conf_dd_confirmed_dispatch_date) and (pd.notnull(expected_dd_expected_to_be_dispatched) or pd.isnull(expected_dd_expected_to_be_dispatched)):
        return 'backorders'
    else:
        return ''

# Apply the function to create the new list_of_backorders column
df['list_of_backorders'] = df.apply(determine_backorders, axis=1)

# Filter out 'Rejected' items when searching for backorders
filtered_df = df[(df['status'] == 'backorder') & (df['list_of_backorders'] == 'backorders')]

# Get the list of DN numbers from the DataFrame
dn_numbers = df['DN no.'].dropna().astype(str).tolist()

# Search for tracking numbers in emails
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Update the existing entries with the tracking numbers
df['Tracking No'] = df['DN no.'].map(tracking_dict)

# Create the tracking details column
df['tracking_details'] = df['Tracking No'].apply(lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else '')

# Set tracking_details to blank if the value contains 'nan'
df['tracking_details'] = df['tracking_details'].apply(lambda x: '' if 'nan' in x else x)

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")

# Close the connection and logout
mail.close()
mail.logout()


Enter the customer info: Girne
Customer info Details:
       ±     gATP  Code Cust. No.  Cust. order no.              Part number  \
5     ⊞   Yellow         CY822025         43931734          N 000000 004039   
122   ⊞    Green         CY822025         43931734           A 447 420 6600   
123   ⊞    Green         CY822025         43931734           A 447 420 6700   
124   ⊞    Green         CY822025         43931734           A 654 094 1500   
133   ⊞    Green         CY822025         43931734           A 002 990 2017   
147   ⊞    Green         CY822025         43931734           A 906 540 1517   
155   ⊞    Green         CY822025         43931734  A 247 584 3313     Z102   
156   ⊞      Red         CY822025         43931734  A 206 584 6505     Z102   
157   ⊞      Red         CY822025         43931734           A 243 814 0000   
158   ⊞    Green         CY822025         43931734  A 177 680 8002     9G33   
159   ⊞      Red         CY822025         43931734  A 247 680 0203     9G33  

('BYE', [b'LOGOUT Requested'])

**İnclusion of mercedes yedek parca file 2025  for correlation  06/01/2025**

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (36).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (36).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'
file_path2025 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2025.xlsx'
input_file = '/content/cgniv_StarOrderplus_Order_items.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'
exdf1 = '/content/df1_unique.xlsx'
exdf2 = '/content/df2.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
if 'Customer info' in df.columns:
    # Updated regex to handle various suffixes including special characters
    df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
else:
    print("The column 'Customer info' was not found in the file.")
    df['Cleaned Customer info'] = df['Customer info']

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1_2024 = pd.read_excel(file_path1)
df1_2025 = pd.read_excel(file_path2025)
df2 = pd.read_excel(file_path2)

# Combine 2024 and 2025 files for lookup
df1_combined = pd.concat([df1_2024, df1_2025], ignore_index=True)

# Check for required columns in df1_combined and df2
required_columns = {'Customer info', 'Part number'}
missing_columns_df1 = required_columns - set(df1_combined.columns)
missing_columns_df2 = required_columns - set(df2.columns)

if missing_columns_df1:
    print(f"The following required columns are missing in the combined file: {missing_columns_df1}")
if missing_columns_df2:
    print(f"The following required columns are missing in the second file: {missing_columns_df2}")

if not missing_columns_df1 and not missing_columns_df2:
    # Clean 'Part number' columns in df1_combined and df2
    df1_combined['Cleaned Part number'] = df1_combined['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
    df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

    # Clean 'Customer info' columns in df1_combined
    df1_combined['Cleaned Customer info'] = df1_combined['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)
    df1_unique = df1_combined.drop_duplicates(subset=['Cleaned Customer info', 'Cleaned Part number'])

    # Save df1_unique for verification
    df1_unique.to_excel(exdf1, index=False)

    # Clean 'Customer info' columns in df2
    df2['Cleaned Customer info'] = df2['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

    # Merge 'StockEntryDate' from df1_unique into df2 based on cleaned columns
    merged_df = pd.merge(df2, df1_unique[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
                         on=['Cleaned Customer info', 'Cleaned Part number'], how='left')

    # Save the resulting DataFrame
    merged_df.to_excel(input_file, index=False)
    print(f"Merged data has been saved to {input_file}")
    print(f"Number of rows in the original df2 file: {len(df2)}")
    print(f"Number of rows in the result file: {len(merged_df)}")

else:
    print("One or both required columns 'Customer info' or 'Part number' were not found in the provided files.")

# Load the updated DataFrame
df = pd.read_excel(input_file)

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            conf_dd_confirmed_dispatch_date = pd.to_datetime(row['Conf. DD confirmed dispatch date'], dayfirst=True, errors='coerce')
            expected_dd_expected_to_be_dispatched = pd.to_datetime(row['Expected DD expecte to be dispatched'], dayfirst=True, errors='coerce')

            if row['Dist. ch.'] == 'af VOR direct':
                days_to_add = 10
            elif row['Dist. ch.'] == 'VOR route':
                days_to_add = 21
            else:
                days_to_add = 10

            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
df['expectedstockentrydate'] = df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
df['status'] = df.apply(determine_status, axis=1)

# Update status for rejected items
df.loc[(df['Items status'] == 'Rejected') & (df['status'] == 'backorder'), 'status'] = 'Rejected'

# Set StockEntryDate to NaT where status is 'backorder' or 'Rejected'
df.loc[df['status'].isin(['backorder', 'Rejected']), 'StockEntryDate'] = pd.NaT

# Save results to a new Excel file
df.to_excel(output_file, index=False)
print(f"Updated file saved to {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/updated_Mercedes_Yedek_Parça_Sipariş_2025.xlsx'

**19122024 trial( it works, but use the one above )**

In [ ]:
import pandas as pd
from datetime import timedelta
import imaplib
import email
from email.header import decode_header
import chardet
import re

# Email account credentials
username = 'kombosawb@gmail.com'
password = 'kyka ypey hfar rjvg'  # Use the App Password here

# File paths for input Excel files
file_path = '/content/updated_Order_items (34).xlsx'
file_path1 = '/content/updated_Mercedes_Yedek_Parça_Sipariş_2024 (34).xlsx'
file_path2 = '/content/niv_StarOrderplus_Order_items_new.xlsx'

# File path for the output Excel file
output_file = '/content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx'

# Load and clean the first Excel file
df = pd.read_excel(file_path)

# Create 'Cleaned Customer info' column by removing spaces, unwanted characters, and suffixes like '-i', '/G', etc.
df['Cleaned Customer info'] = df['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

# Remove rows with unwanted 'Customer info' entries
df_cleaned = df[~df['Cleaned Customer info'].str.lower().str.startswith(('stok', 'stpk', 'minarelikoy', 'mýna', 'stock', 'raf')) &
                ~df['Cleaned Customer info'].str.startswith(('RAF', 'STOCKMT'))]

# Save the cleaned DataFrame to a new Excel file
df_cleaned.to_excel(file_path2, index=False)

# Load the cleaned DataFrame and other Excel files
df_cleaned = pd.read_excel(file_path2)
df1 = pd.read_excel(file_path1)
df2 = pd.read_excel(file_path2)

# Clean 'Part number' columns in df1 and df2
df1['Cleaned Part number'] = df1['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]
df2['Cleaned Part number'] = df2['Part number'].astype(str).str.replace(' ', '').str.upper().str[:10]

# Clean 'Customer info' columns in df1
df1['Cleaned Customer info'] = df1['Customer info'].astype(str).str.replace(' ', '').str.upper().str.replace(r'[-/][A-ZİÝIi]+$', '', regex=True)

# Handle missing StockEntryDate by propagating the value from duplicates
def fill_missing_stockentrydate(group):
    """
    Fill missing StockEntryDate for duplicate rows within the same group.
    If one row in the group has a StockEntryDate, propagate it to others.
    """
    if group['StockEntryDate'].notna().any():
        group['StockEntryDate'] = group['StockEntryDate'].fillna(method='bfill').fillna(method='ffill')
    return group

# Apply the function to fill missing StockEntryDates in Mercedes Yedek Parça file
df1 = df1.groupby(['Cleaned Customer info', 'Cleaned Part number'], group_keys=False).apply(fill_missing_stockentrydate)

# Merge 'StockEntryDate' from Mercedes Yedek Parça into Star Order file
merged_df = pd.merge(
    df2,
    df1[['Cleaned Customer info', 'Cleaned Part number', 'StockEntryDate']],
    on=['Cleaned Customer info', 'Cleaned Part number'],
    how='left'
)

# Save the resulting DataFrame
merged_df.to_excel(file_path2, index=False)

# Define the function to compute the expected stock entry date
def calculate_expected_stock_entry_date(row):
    try:
        inv_date_dispatch_date = pd.to_datetime(row['Inv. date dispatch date'], dayfirst=True, errors='coerce')
        if pd.notnull(inv_date_dispatch_date):
            days_to_add = 10 if row['Dist. ch.'] in ['af VOR direct', 'VOR route'] else 21
            return inv_date_dispatch_date + timedelta(days=days_to_add)
        return pd.NaT
    except KeyError as e:
        print(f"Column not found: {e}")
        return pd.NaT

# Apply the function to each row to create the new column
merged_df['expectedstockentrydate'] = merged_df.apply(calculate_expected_stock_entry_date, axis=1)

# Define the function to compute the status
def determine_status(row):
    inv_date_dispatch_date = pd.to_datetime(row.get('Inv. date dispatch date', None), dayfirst=True, errors='coerce')
    conf_dd_confirmed_dispatch_date = pd.to_datetime(row.get('Conf. DD confirmed dispatch date', None), dayfirst=True, errors='coerce')
    expected_dd_expected_to_be_dispatched = pd.to_datetime(row.get('Expected DD expecte to be dispatched', None), dayfirst=True, errors='coerce')

    if pd.notnull(inv_date_dispatch_date):
        return f'invoice dispatch date on {inv_date_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(conf_dd_confirmed_dispatch_date):
        return f'confirmed dispatch date on {conf_dd_confirmed_dispatch_date.strftime("%d/%m/%Y")}'
    elif pd.notnull(expected_dd_expected_to_be_dispatched):
        return 'backorder'
    else:
        return 'backorder'

# Apply the function to create the new status column
merged_df['status'] = merged_df.apply(determine_status, axis=1)

# Update status for rejected items
merged_df.loc[(merged_df['Items status'] == 'Rejected') & (merged_df['status'] == 'backorder'), 'status'] = 'Rejected'

# Filter out 'Rejected' items when searching for backorders
filtered_df = merged_df[(merged_df['status'] == 'backorder')]

# Fetch tracking numbers from emails
def fetch_stt_number(body):
    stt_number_match = re.search(r'STT Number[:\.\s]+(\d+)', body)
    return stt_number_match.group(1) if stt_number_match else 'Not Found'

def search_emails_for_tracking_numbers(dn_numbers):
    tracking_dict = {dn_number: 'Not Found' for dn_number in dn_numbers}
    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    mail.login(username, password)
    mail.select('inbox')

    result, data = mail.search(None, 'ALL')
    if result != 'OK':
        print("Failed to search for emails.")
        return tracking_dict

    email_ids = data[0].split()
    for email_id in email_ids:
        result, msg_data = mail.fetch(email_id, '(RFC822)')
        if result != 'OK':
            continue

        raw_email = msg_data[0][1]
        msg = email.message_from_bytes(raw_email)

        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True)
                    detected_encoding = chardet.detect(payload)['encoding']
                    body = payload.decode(detected_encoding or 'utf-8', errors='replace')
                    break
        else:
            payload = msg.get_payload(decode=True)
            detected_encoding = chardet.detect(payload)['encoding']
            body = payload.decode(detected_encoding or 'utf-8', errors='replace')

        for dn_number in dn_numbers:
            if dn_number in body:
                tracking_dict[dn_number] = fetch_stt_number(body)

    mail.close()
    mail.logout()
    return tracking_dict

dn_numbers = merged_df['DN no.'].dropna().astype(str).tolist()
tracking_dict = search_emails_for_tracking_numbers(dn_numbers)

# Update with tracking numbers and links
merged_df['Tracking No'] = merged_df['DN no.'].map(tracking_dict)
merged_df['tracking_details'] = merged_df['Tracking No'].apply(
    lambda x: f'https://dbschenker.com/app/tracking-public/?refNumber={x}' if x != 'Not Found' else ''
)

# Save the final output
merged_df.to_excel(output_file, index=False)
print(f"Final updated file saved to {output_file}")


<ipython-input-3-613ec88e546f>:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['StockEntryDate'] = group['StockEntryDate'].fillna(method='bfill').fillna(method='ffill')
<ipython-input-3-613ec88e546f>:53: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group['StockEntryDate'] = group['StockEntryDate'].fillna(method='bfill').fillna(method='ffill')
<ipython-input-3-613ec88e546f>:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns af

Final updated file saved to /content/vaugzcstcketrd_expstckentryd_status_trcklnk_trckno.xlsx
